In [ ]:
import json
import itertools
import pandas as pd
import numpy as np
from tqdm import tqdm

## Functions

In [ ]:
def statistical_distance(distributionA, distributionB, n):
    """
    Calculate the statistical distance between two probability distributions on `n`-bit strings.
    
    Args:
        distribution{A,B} (Dict[str, float]): keys are bitstrings and values are probabilities.
    
    """
    total = 0
    for bits in itertools.product([0, 1], repeat=n):
        bit_string = ''.join(map(str, bits))
        try:
            total += abs(distributionA[bit_string] - distributionB[bit_string])
        except KeyError:
            total += distributionB[bit_string]
    total = total / 2
    return total

In [ ]:
def xor_bitstrings(bitstring1, bitstring2):
    """
    Bitwise XOR of two bitstrings.
    
    """
    result = ""
    for i in range(len(bitstring1)):
        current_xor = int(bitstring1[i]) ^ int(bitstring2[i])
        result += str(current_xor)
    return result

In [ ]:
def counts_to_probabilities(data_dict):
    """
    Convert dictionary values from counts to probabilities.
    
    """
    total = 0
    for v in data_dict.values():
        total += v
    out = {k : v / total for k,v in data_dict.items()}
    return out

In [ ]:
def little_to_big_endian(data_dict):
    """
    Convert dictionary keys from little to big endian.
    
    """
    out_dict = {}
    for k,v in data_dict.items():
        out_dict[k[::-1]] = v
    return out_dict

In [ ]:
def post_select(data_dict, valid_states):
    """
    Remove dictionary items whose keys are not contained in `valid_states`.
    
    """
    post_data = {}
    n = 0
    n_post = 0
    for k,v in data_dict.items():
        n += v
        if k in valid_states:
            post_data[k] = v
            n_post += v
    return post_data, n_post/n  

In [ ]:
def logical_states(seed, stabilizers):
    """
    Return a bit-string representation of `stabilizers` applied to the state specified by `seed`.
    
    Args:
        seed (str) : bit-string representation of the seed state.
        stabilizers (List[str]) : list of stabilizers represented as bit-strings.
    
    """
    states = []
    for combo in itertools.chain.from_iterable(itertools.combinations(stabilizers, r) for r in range(len(stabilizers)+1)):
        if len(combo) == 0:
            state = seed
        elif len(combo) == 1:
            state = xor_bitstrings(seed, combo[0])
        elif len(combo) == 2:
            state = xor_bitstrings(xor_bitstrings(seed, combo[0]), combo[1])
        elif len(combo) == 3:
            state = xor_bitstrings(xor_bitstrings(xor_bitstrings(seed, combo[0]), combo[1]), combo[2])
        elif len(combo) == 4:
            state = xor_bitstrings(xor_bitstrings(xor_bitstrings(xor_bitstrings(seed, combo[0]), combo[1]), combo[2]), combo[3])
        states += [state]
    return states

In [ ]:
def physical_to_logical(physical, states, n):
    """
    Map dictionary values from physical states to logical states.
    
    Args:
        physical (Dict[str, numeric]): keys are physical states, values are counts/probabilities.
        states (Dict[str, List[str]]): map from logical states to physical states
        n (int) : number of bits in the logical state representation.
        
    """
    logical = {}
    for bits in itertools.product([0, 1], repeat=n):
        bit_string = ''.join(map(str, bits))
        logical[bit_string] = 0
    for k, v in physical.items():
        for bit_string in logical.keys():
            if k in states[bit_string]:
                logical[bit_string] += v
    return logical

In [ ]:
def dict_to_df(data_dict):
    """
    Convert dataframe into dict.
    
    """
    df = pd.DataFrame()
    for k, v in data_dict.items():
        for i in range(int(v)):
            df = pd.concat([df, pd.DataFrame([[k, 1]], columns=['result', 'counts'])], ignore_index=True)
    return df

In [ ]:
def df_to_dict(df):
    """
    Convert dict into dataframe
    
    """
    data_dict = {}
    keys = df['result'].unique()
    for k in keys:
        data_dict[k] = df[df['result'] == k].shape[0]
    return data_dict

## Expected distributions

In [ ]:
bits = ['0', '1']
gates = [''.join(x) for x in itertools.product(bits, repeat=4)]

In [ ]:
expected_ghz_distribution = {
    '000' : 0,
    '001' : .25,
    '010' : .25,
    '011' : 0,
    '100' : .25,
    '101' : 0,
    '110' : 0,
    '111' : .25,
}

In [ ]:
expected_ghz_distributions_z = {
    g : {
        '000' : .5,
        '001' : 0,
        '010' : 0,
        '011' : 0,
        '100' : 0,
        '101' : 0,
        '110' : 0,
        '111' : .5,
    }
    for g in gates
}

In [ ]:
expected_ppp_distributions_z = {
    g : {
        '000' : .125,
        '001' : .125,
        '010' : .125,
        '011' : .125,
        '100' : .125,
        '101' : .125,
        '110' : .125,
        '111' : .125,
    }
    for g in gates
}

In [ ]:
expected_ghz_distributions_x = {
    '0000' : {
        '000' : .25,
        '001' : 0,
        '010' : 0,
        '011' : .25,
        '100' : 0,
        '101' : .25,
        '110' : .25,
        '111' : 0,
    },
    '1000' : {
        '000' : 0,
        '001' : .25,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '0100' : {
        '000' : 0,
        '001' : .25,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '0010' : {
        '000' : 0,
        '001' : .25,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '0001' : {
        '000' : 0,
        '001' : .25,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '1100' : {
        '000' : .25,
        '001' : 0,
        '010' : 0,
        '011' : .25,
        '100' : 0,
        '101' : .25,
        '110' : .25,
        '111' : 0,
    },
    '0110' : {
        '000' : .25,
        '001' : 0,
        '010' : 0,
        '011' : .25,
        '100' : 0,
        '101' : .25,
        '110' : .25,
        '111' : 0,
    },
    '1010' : {
        '000' : .25,
        '001' : 0,
        '010' : 0,
        '011' : .25,
        '100' : 0,
        '101' : .25,
        '110' : .25,
        '111' : 0,
    },
    '1001' : {
        '000' : .25,
        '001' : 0,
        '010' : 0,
        '011' : .25,
        '100' : 0,
        '101' : .25,
        '110' : .25,
        '111' : 0,
    },
    '0101' : {
        '000' : .25,
        '001' : 0,
        '010' : 0,
        '011' : .25,
        '100' : 0,
        '101' : .25,
        '110' : .25,
        '111' : 0,
    },
    '0011' : {
        '000' : .25,
        '001' : 0,
        '010' : 0,
        '011' : .25,
        '100' : 0,
        '101' : .25,
        '110' : .25,
        '111' : 0,
    },
    '1101' : {
        '000' : 0,
        '001' : .25,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '1011' : {
        '000' : 0,
        '001' : .25,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '0111' : {
        '000' : 0,
        '001' : .25,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '1110' : {
        '000' : 0,
        '001' : .25,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '1111' : {
        '000' : .25,
        '001' : 0,
        '010' : 0,
        '011' : .25,
        '100' : 0,
        '101' : .25,
        '110' : .25,
        '111' : 0,
    }
}

In [ ]:
expected_ppp_distributions_x = {
    '0000' : {
        '000' : 1,
        '001' : 0,
        '010' : 0,
        '011' : 0,
        '100' : 0,
        '101' : 0,
        '110' : 0,
        '111' : 0,
    },
    '0001' : {
        '000' : .5625,
        '001' : .0625,
        '010' : .0625,
        '011' : .0625,
        '100' : .0625,
        '101' : .0625,
        '110' : .0625,
        '111' : .0625,
    },
    '0010' : {
        '000' : .25,
        '001' : 0,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : .25,
        '111' : 0,
    },
    '0100' : {
        '000' : .25,
        '001' : .25,
        '010' : 0,
        '011' : 0,
        '100' : .25,
        '101' : .25,
        '110' : 0,
        '111' : 0,
    },
    '1000' : {
        '000' : .25,
        '001' : .25,
        '010' : .25,
        '011' : .25,
        '100' : 0,
        '101' : 0,
        '110' : 0,
        '111' : 0,
    },
    '0011' : {
        '000' : .5625,
        '001' : .0625,
        '010' : .0625,
        '011' : .0625,
        '100' : .0625,
        '101' : .0625,
        '110' : .0625,
        '111' : .0625,
    },
    '0101' : {
        '000' : .5625,
        '001' : .0625,
        '010' : .0625,
        '011' : .0625,
        '100' : .0625,
        '101' : .0625,
        '110' : .0625,
        '111' : .0625,
    },
    '1001' : {
        '000' : .5625,
        '001' : .0625,
        '010' : .0625,
        '011' : .0625,
        '100' : .0625,
        '101' : .0625,
        '110' : .0625,
        '111' : .0625,
    },
    '1100' : {
        '000' : .25,
        '001' : .25,
        '010' : 0,
        '011' : 0,
        '100' : 0,
        '101' : 0,
        '110' : .25,
        '111' : .25,
    },
    '1010' : {
        '000' : .25,
        '001' : 0,
        '010' : .25,
        '011' : 0,
        '100' : 0,
        '101' : .25,
        '110' : 0,
        '111' : .25,
    },
    '0110' : {
        '000' : .25,
        '001' : 0,
        '010' : 0,
        '011' : .25,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '1110' : {
        '000' : 0,
        '001' : .25,
        '010' : .25,
        '011' : 0,
        '100' : .25,
        '101' : 0,
        '110' : 0,
        '111' : .25,
    },
    '1011' : {
        '000' : .0625,
        '001' : .0625,
        '010' : .5625,
        '011' : .0625,
        '100' : .0625,
        '101' : .0625,
        '110' : .0625,
        '111' : .0625,
    },
    '0111' : {
        '000' : .0625,
        '001' : .0625,
        '010' : .0625,
        '011' : .0625,
        '100' : .5625,
        '101' : .0625,
        '110' : .0625,
        '111' : .0625,
    },
    '1101' : {
        '000' : .0625,
        '001' : .5625,
        '010' : .0625,
        '011' : .0625,
        '100' : .0625,
        '101' : .0625,
        '110' : .0625,
        '111' : .0625,
    },
    '1111' : {
        '000' : .0625,
        '001' : .0625,
        '010' : .0625,
        '011' : .0625,
        '100' : .0625,
        '101' : .0625,
        '110' : .0625,
        '111' : .5625,
    }
}

In [ ]:
expected_distributions_dict = {
    '+++' : {
        'x' : expected_ppp_distributions_x,
        'z' : expected_ppp_distributions_z
    },
    'ghz' : {
        'x' : expected_ghz_distributions_x,
        'z' : expected_ghz_distributions_z
    }
}

## $[\![8,3,2]\!]$ states

In [ ]:
n = 8

In [ ]:
valid_states_832_x = [''.join(x) for x in itertools.product(bits, repeat=n) if x.count('1') % 2 == 0]
stabilizers_z = ['11110000', '00001111', '10101010', '11001100']
states_map_832_x = {
    '000' : logical_states('00000000', stabilizers_z),
    '001' : logical_states('10001000', stabilizers_z),
    '010' : logical_states('10100000', stabilizers_z),
    '011' : logical_states('00101000', stabilizers_z),
    '100' : logical_states('11000000', stabilizers_z),
    '101' : logical_states('01001000', stabilizers_z),
    '110' : logical_states('01100000', stabilizers_z),
    '111' : logical_states('11101000', stabilizers_z),
}
sorted(valid_states_832_x) == sorted([item for sublist in states_map_832_x.values() for item in sublist])

In [ ]:
stabilizers_x = ['11111111']
states_map_832_z = {
    '000' : logical_states('00000000', stabilizers_x),
    '001' : logical_states('10101010', stabilizers_x),
    '010' : logical_states('11001100', stabilizers_x),
    '011' : logical_states('01100110', stabilizers_x),
    '100' : logical_states('11110000', stabilizers_x),
    '101' : logical_states('01011010', stabilizers_x),
    '110' : logical_states('00111100', stabilizers_x),
    '111' : logical_states('10010110', stabilizers_x),
}
valid_states_832_z = [item for sublist in states_map_832_z.values() for item in sublist]

In [ ]:
states_map_832_dict = {'x' : states_map_832_x, 'z' : states_map_832_z}
valid_states_832_dict = {'x' : valid_states_832_x, 'z' : valid_states_832_z}

## Build dataframe

In [ ]:
experiments = [
    {'qpu' : 'ionq-11q', 'date' : '2023-08-17'},
    {'qpu' : 'ionq-11q', 'date' : '2023-07-26'},
    {'qpu' : 'ibmq_mumbai', 'date' : '2023-04-14'},
]
resamplings = 100

df_all = pd.DataFrame()

for experiment in experiments:
    with open(f'data-{experiment["qpu"]}-{experiment["date"]}.json', 'r') as f:
        data = json.load(f)
    
    for sp in ['+++', 'ghz']:
        for meas in ['x', 'z']:
            states_map_832 = states_map_832_dict[meas]
            valid_states_832 = valid_states_832_dict[meas]
            statistical_distances = []
            errs = []
            post_selection_rates = []
            encodings = []
            for g in tqdm(gates):
                try:
                    distances = []
                    expected_distributions = expected_distributions_dict[sp][meas]
                    # The df conversion is slow so we do it outside the loop
                    df = dict_to_df(data[f'bare_prep={sp}_gate={g}_readout={meas}'])
                    # print(df)
                    for i in range(resamplings):
                        bare_distribution = counts_to_probabilities(df_to_dict(df.sample(frac=1, replace=True)))
                        distances += [statistical_distance(bare_distribution, expected_distributions[g], 3)]
                    statistical_distances += [np.mean(distances)]
                    errs += [np.std(distances)]
                    post_selection_rates += [1]
                    if g == '0000':
                        encodings += ['bare']
                except KeyError:
                    pass
                
                try:
                    distances = []
                    post_rates = []
                    df = dict_to_df(little_to_big_endian(data[f'enc_prep={sp}_gate={g}_readout={meas}']))
                    for i in range(resamplings):
                        post_data, post_rate = post_select(df_to_dict(df.sample(frac=1, replace=True)), valid_states_832)
                        enc_distribution = counts_to_probabilities(physical_to_logical(post_data, states_map_832, 3))
                        distances += [statistical_distance(enc_distribution, expected_distributions[g], 3)]
                        post_rates += [post_rate]
                    statistical_distances += [np.mean(distances)]
                    errs += [np.std(distances)]
                    post_selection_rates += [np.mean(post_rates)]
                    if g == '0000':
                        encodings += ['[[8,3,2]]']
                except KeyError:
                    pass
                
                try:
                    distances = []
                    post_rates = []
                    df = dict_to_df(little_to_big_endian(data[f'encf_prep={sp}_gate={g}_readout={meas}']))
                    valid_states_832_flag = []
                    for state in valid_states_832:
                        flag_state = state + "000"
                        valid_states_832_flag += [flag_state]
                    for i in range(resamplings):
                        post_data, post_rate = post_select(df_to_dict(df.sample(frac=1, replace=True)), valid_states_832_flag)
                        new_post_data = {}
                        for k in post_data.keys():
                            new_post_data[k[0:8]] = post_data[k]
                        enc_distribution = counts_to_probabilities(physical_to_logical(new_post_data, states_map_832, 3))
                        distances += [statistical_distance(enc_distribution, expected_distributions[g], 3)]
                        post_rates += [post_rate]
                    statistical_distances += [np.mean(distances)]
                    errs += [np.std(distances)]
                    post_selection_rates += [np.mean(post_rates)]
                    if g == '0000':
                        encodings += ['[[8,3,2]] flag']
                except KeyError:
                    pass
            
            if len(errs) > 0:
                df = pd.DataFrame.from_dict({
                    'date' : [experiment['date']] * len(errs),
                    'qpu' : [experiment['qpu']] * len(errs),
                    'state-prep' : [sp] * len(errs),
                    'readout' : [meas] * len(errs),
                    'encoding' : encodings * (len(errs) // len(encodings)),
                    'gate' : [item[::-1] for sublist in [[g] * (len(errs) // len(gates)) for g in gates] for item in sublist],
                    'distance': statistical_distances,
                    'error': errs,
                    'post-select': post_selection_rates
                })
                df = df.sort_values(by=['gate', 'encoding'])
                df_all = pd.concat([df_all, df], ignore_index=True)
            
df_all.to_csv('results.csv', index=False)

In [ ]:
df_all